In [ ]:
### Resampling Dataset
# Show Value Counts
display(round(df['DEATH_EVENT'].value_counts() / len(df)  * 100, 1))
Resampling Observations-
* We can see that we don't need to have resampling done to this dataframe. But I decided to experiment and see how this would come out.
# Resample the DataSet
X_resamp, y_resamp = RandomOverSampler(random_state=0).fit_resample(X, y)
X_train_resamp, y_train_resamp = RandomOverSampler(random_state=0).fit_resample(X_train, y_train)
X_test_resamp, y_test_resamp = RandomOverSampler(random_state=0).fit_resample(X_test, y_test)

# Run Classification Metrics
classifierMetrics(classifiers=classifier_list,   # List of classifiers
                  classifier_name=class_name,    # Classifier names
                  X=X_resamp,                    # X data
                  y=y_resamp,                    # Y data
                  X_train=X_train_resamp,        # X_train Dataset
                  y_train=y_train_resamp,        # y_train Dataset
                  X_test=X_test_resamp,          # X_test Dataset
                  y_test=y_test_resamp,          # y_test Dataset
                  kfolds=5)                      # of K-folds 
ReSampling Obeservations-
* Resampling was applied and we can see an increase in F1-score which is great.
* We see a slight decrease in accuracy, but when we compare our results to the testing results we can see that there is a smaller difference between the values. So this is something that is great. 
##### Ensemble Random Sampling
# Top 3 Classifiers in Results Testing
voting_clf_resampled = VotingClassifier(estimators=[ ('Random Forest', RandomForestClassifier(random_state=0)),   # 1 Classifier
                                                     ('ADA Boost', AdaBoostClassifier(random_state=0)),           # 2 Classifier
                                                     ('XGB', XGBClassifier(random_state=0))])                     # 3 Classifier

# Train Voting Classifier
voting_clf_resampled.fit(X_train, y_train)

# Change Classification to Soft for prediction Probabilties
voting_clf_resampled.voting = 'soft'

# Add this model to classifier list
classifier_list.append(voting_clf_resampled)
class_name.append('Resampled Voting Classifier')

# Run Classification Metrics
classifierMetrics(classifiers=classifier_list,   # List of classifiers
                  classifier_name=class_name,    # Classifier names
                  X=X_resamp,                   # X data
                  y=y_resamp,                   # Y data
                  X_train=X_train_resamp,       # X_train Dataset
                  y_train=y_train_resamp,       # y_train Dataset
                  X_test=X_test_resamp,         # X_test Dataset
                  y_test=y_test_resamp,         # y_test Dataset
                  kfolds=5)                     # of K-folds 
Ensemble Random Sampling Observations-
* We can see that this does perform well, but not the best. So we are going to continue with our Random Forest Classifier.
##### Hypertuned Random Sampling
# Create the parameter grid based on the results of random search 
param_grid = {
    'random_state': [0,5,10],
    'max_depth': [5, 20, 50, 100],
    'max_features': [2, 3, 5],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [30, 50, 150, 200]
}

# Create a based model
rf_model = RandomForestClassifier(random_state=0)

# Instantiate the grid search model
optimized_RandomForest = GridSearchCV(estimator = rf_model, 
                           param_grid = param_grid, 
                           cv = 3, 
                           n_jobs = -1,
                           scoring='f1')

# Fit the grid search to the data
optimized_RandomForest.fit(X_train_resamp, y_train_resamp)

# Print Optimal Parameters
print('Optimized Parameters')
print(optimized_RandomForest.best_params_)
# Add this model to classifier list
classifier_list.append(RandomForestClassifier(max_depth=20, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=150, random_state=5))
class_name.append('Optimized Resampled Random Forest')

# Run Classification Metrics
classifierMetrics(classifiers=classifier_list,   # List of classifiers
                  classifier_name=class_name,    # Classifier names
                  X=X_resamp,                    # X data
                  y=y_resamp,                    # Y data
                  X_train=X_train_resamp,        # X_train Dataset
                  y_train=y_train_resamp,        # y_train Dataset
                  X_test=X_test_resamp,          # X_test Dataset
                  y_test=y_test_resamp,          # y_test Dataset
                  kfolds=5)                      # of K-folds 
Hypertuning Random Sampling Observations-
* We can see that this hypertuned model does not perform better. So we will be sticking Random Forest Classification.
# Show Importance Information
treeRegressionImportance(model=RandomForestClassifier(random_state=0),   # Best performing regression model
                         X_train=X_train_resamp,                         # X_train data
                         y_train=y_train_resamp,                         # y_train data
                         model_name='Random Forest Regression')          # Name of Tree Regression model
Resampled Feature Importance Obeservaitions-
* We can see that time is the largest predictor in this model. 
* We can see that there is importance of all features in this model. There are no features that are dropping off at the end. 